# Validation of PCM and VBR models against each other with DP SMIB Load Step

## Load Libraries

In [ ]:
import re
import numpy as np
import math
import os
import subprocess
import pickle
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

#%matplotlib widget

## Simulation parameters

In [ ]:
root_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
path_exec = root_path + '/build/dpsim/examples/cxx/'

name_executable_vbr = 'DP_SMIB_ReducedOrderSG_LoadStep'
name_vbr = "DP_SMIB_ReducedOrderSG_VBR_LoadStep"

name_executable_pcm = 'DP_SMIB_ReducedOrderSGIterative_LoadStep'
name_pcm = "DP_SMIB_ReducedOrderSGIterativePCM_LoadStep"

# times in s
end_time = 1
load_step_time = 0.1
roi_begin = 0.1
roi_end = 1

# config params
timestep_vbr = 0.1e-3
timestep_pcm = 10e-6
max_iter = 10
tolerance = 1e-6

roi_begin_idx = int(roi_begin/timestep_pcm)
roi_end_idx = int(roi_end/timestep_pcm)

logs_path = 'logs'
if not os.path.exists(logs_path):
    os.mkdir(logs_path)
    
var_name = 'SynGen.Te'

te_ref = 0.5454986888690558

## Parametrization

In [ ]:
from math import sqrt, pi

### Power System
nominal_voltage_hv = 230e3
nominal_voltage_mv = 24e3
ratio = nominal_voltage_mv/nominal_voltage_hv
frequency = 60
omega = 2 * pi * frequency

### Generator
nom_power = 555e6
set_point_active_power = 300e6
set_point_voltage = 1.05*nominal_voltage_mv
H = 3.7
Td0_t = 8.0669
Td0_s = 0.0300
Td_t = 1.3368
Td_s = 0.0230
Ld_t = 0.2999
Ld_s = 0.2299
Tq0_t = 0.9991
Tq0_s = 0.0700
Lq_t = 0.6500
Lq_s = 0.2500
Ld = 1.8099
Lq = 1.7600
L0 = 0.15
Taa = 0
      
### PiLine parameters calculated from CIGRE Benchmark system
line_length = 100
line_resistance = 1.27e-4 * 529 * line_length * pow(ratio,2)
line_inductance = 9.05e-4 * 529 / omega * line_length * pow(ratio,2)
line_capacitance = (1.81e-3 / 529) / omega * line_length / pow(ratio,2)
line_conductance = 0.0048

### Load step
load_step_activePower = 100e6; 

## Run powerflow for Initialization

In [ ]:
sim_name_pf = "DP_SMIB_ReducedOrderSGIterative_LoadStep_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

### Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf  = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

### Components

# syncrhon generator
gen_pf = dpsimpy.sp.ph1.SynchronGenerator('gen', dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=nom_power, rated_voltage=nominal_voltage_mv, 
                      set_point_active_power=set_point_active_power, set_point_voltage=set_point_voltage, 
                      powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(nominal_voltage_mv)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(nominal_voltage_mv)
extnet_pf.set_base_voltage(nominal_voltage_mv)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# PiLine
pi_line_pf = dpsimpy.sp.ph1.PiLine('Pi_Line_pf', dpsimpy.LogLevel.debug)
pi_line_pf.set_parameters(R=line_resistance, L=line_inductance, C=line_capacitance, G=line_conductance)
pi_line_pf.set_base_voltage(nominal_voltage_mv)

### Connections
gen_pf.connect([n1_pf])
pi_line_pf.connect([n2_pf, n1_pf])
extnet_pf.connect([n2_pf])

### Define system topology
system_pf = dpsimpy.SystemTopology(frequency, [n1_pf, n2_pf], [gen_pf, pi_line_pf, extnet_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)
logger_pf.log_attribute('p_inj', 'p_inj', extnet_pf)
logger_pf.log_attribute('q_inj', 'q_inj', extnet_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

## DPSim Topology for DP simulations

In [ ]:
def dp_reducedOrderSG_loadStep(sim_pf, gen_pf, gen_model="4TPM", event_time=0.1, final_time=0.5, time_step=10e-6, max_iter=10, tolerance=1e-6):
    
    ### DPsim DP simulation
    name = "DP_SMIB_ReducedOrderSGIterative_LoadStep_" + gen_model
    dpsimpy.Logger.set_log_dir("logs/" + name)

    ### Nodes
    gnd = dpsimpy.dp.SimNode.gnd
    n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single) 
    n2 = dpsimpy.dp.SimNode('n2', dpsimpy.PhaseType.Single)

    ### Components

    # syncrhon generator
    gen = None
    if (gen_model=="4VBR"):
        gen = dpsimpy.dp.ph1.SynchronGenerator4OrderVBR('gen', dpsimpy.LogLevel.debug)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t)		
    elif (gen_model=="6VBR"):
        gen = dpsimpy.dp.ph1.SynchronGenerator6bOrderVBR('gen', dpsimpy.LogLevel.debug)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t,
                                                Ld_s=Ld_s, Lq_s=Lq_s, Td0_s=Td0_s, Tq0_s=Tq0_s)
    elif (gen_model=="4TPM"):
        gen = dpsimpy.dp.ph1.SynchronGenerator4OrderTPM('gen', dpsimpy.LogLevel.debug)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t)
        gen.set_max_iterations(max_iter=max_iter)
        gen.set_tolerance(tolerance=tolerance)
    elif (gen_model=="4PCM"):
        gen = dpsimpy.dp.ph1.SynchronGenerator4OrderPCM('gen', dpsimpy.LogLevel.debug)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t)
        gen.set_max_iterations(max_iter=max_iter)
        gen.set_tolerance(tolerance=tolerance)		
    elif (gen_model=="6PCM"):
        gen = dpsimpy.dp.ph1.SynchronGenerator6OrderPCM('gen', dpsimpy.LogLevel.debug)
        gen.set_operational_parameters_per_unit(nom_power=nom_power, nom_voltage=nominal_voltage_mv, nom_frequency=frequency, 
                                                H=H, Ld=Ld, Lq=Lq, L0=L0, Ld_t=Ld_t, Lq_t=Lq_t, Td0_t=Td0_t, Tq0_t=Tq0_t,
                                                Ld_s=Ld_s, Lq_s=Lq_s, Td0_s=Td0_s, Tq0_s=Tq0_s)
        gen.set_max_iterations(max_iter=max_iter)
        gen.set_tolerance(tolerance=tolerance)

    # Switch
    switch = dpsimpy.dp.ph1.Switch('Load_Add_Switch_', dpsimpy.LogLevel.debug)
    resistance = abs(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])**2 / load_step_activePower
    switch.set_parameters(1e9, resistance)
    switch.open()

    # pi line
    pi_line = dpsimpy.dp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
    pi_line.set_parameters(R=line_resistance, L=line_inductance,
                           C=line_capacitance, G=line_conductance)

    # Slack
    slack = dpsimpy.dp.ph1.NetworkInjection('slack', dpsimpy.LogLevel.debug)
    slack.set_parameters(V_ref=nominal_voltage_mv)
    
    ### Connections
    gen.connect([n1])
    switch.connect([gnd, n1])
    pi_line.connect([n1, n2])
    slack.connect([n2])
    
    ### Define system topology
    system = dpsimpy.SystemTopology(frequency, [n1, n2], [gen, pi_line, slack, switch])

    ### Logging
    logger = dpsimpy.Logger(name)
    logger.log_attribute('Te', 'Te', gen)

    # init node voltages and SG power with power flow
    system.init_with_powerflow(systemPF=system_pf)
    
    ### Simulation
    sim = dpsimpy.Simulation(name, dpsimpy.LogLevel.debug)
    sim.set_system(system)
    sim.do_init_from_nodes_and_terminals(True)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    if (gen_model=="4VBR" or gen_model=="6VBR"):
        sim.do_system_matrix_recomputation(True)
    
    sw_event_1 = dpsimpy.event.SwitchEvent(event_time, switch, True)
    sim.add_event(sw_event_1)
    
    sim.add_logger(logger)
    sim.run()
    
    return name

## Run DPSim simulations (DP Domain)

### 4th Order VBR

In [ ]:
log_name = dp_reducedOrderSG_loadStep(sim_pf, gen_pf, gen_model="4VBR", event_time=load_step_time, final_time=end_time, time_step=timestep_vbr)

In [ ]:
# read Simulink log file

file_path = os.getcwd() + "/logs/" + log_name + "/" + log_name + ".csv"
ts_dpsim_4VBR = read_timeseries_dpsim(file_path)
ts_dpsim_4VBR['Te'] = TimeSeries('Te', ts_dpsim_4VBR['Te'].interpolate(timestep_pcm).time, ts_dpsim_4VBR['Te'].interpolate(timestep_pcm).values)
ts_dpsim_4VBR_roi={}
ts_dpsim_4VBR_roi['Te'] = TimeSeries('Te', ts_dpsim_4VBR['Te'].time[roi_begin_idx:roi_end_idx], ts_dpsim_4VBR['Te'].values[roi_begin_idx:roi_end_idx])

### 6th Order VBR

In [ ]:
log_name = dp_reducedOrderSG_loadStep(sim_pf, gen_pf, gen_model="6VBR", event_time=load_step_time, final_time=end_time, time_step=timestep_vbr)

In [ ]:
# read Simulink log file

file_path = os.getcwd() + "/logs/" + log_name + "/" + log_name + ".csv"
ts_dpsim_6VBR = read_timeseries_dpsim(file_path)
ts_dpsim_6VBR['Te'] = TimeSeries('Te', ts_dpsim_6VBR['Te'].interpolate(timestep_pcm).time, ts_dpsim_6VBR['Te'].interpolate(timestep_pcm).values)
ts_dpsim_6VBR_roi={}
ts_dpsim_6VBR_roi['Te'] = TimeSeries('Te', ts_dpsim_6VBR['Te'].time[roi_begin_idx:roi_end_idx], ts_dpsim_6VBR['Te'].values[roi_begin_idx:roi_end_idx])

### 4th Order PCM

In [ ]:
log_name = dp_reducedOrderSG_loadStep(sim_pf, gen_pf, gen_model="4PCM", event_time=load_step_time, final_time=end_time, time_step=timestep_pcm, max_iter=max_iter, tolerance=tolerance)

In [ ]:
# read Simulink log file

file_path = os.getcwd() + "/logs/" + log_name + "/" + log_name + ".csv"
ts_dpsim_4PCM = read_timeseries_dpsim(file_path)
ts_dpsim_4PCM_roi={}
ts_dpsim_4PCM_roi['Te'] = TimeSeries('Te', ts_dpsim_4PCM['Te'].time[roi_begin_idx:roi_end_idx], ts_dpsim_4PCM['Te'].values[roi_begin_idx:roi_end_idx])

### 6th Order PCM

In [ ]:
log_name = dp_reducedOrderSG_loadStep(sim_pf, gen_pf, gen_model="6PCM", event_time=load_step_time, final_time=end_time, time_step=timestep_pcm, max_iter=max_iter, tolerance=tolerance)

In [ ]:
# read Simulink log file
file_path = os.getcwd() + "/logs/" + log_name + "/" + log_name + ".csv"
ts_dpsim_6PCM = read_timeseries_dpsim(file_path)
ts_dpsim_6PCM_roi={}
ts_dpsim_6PCM_roi['Te'] = TimeSeries('Te', ts_dpsim_6PCM['Te'].time[roi_begin_idx:roi_end_idx], ts_dpsim_6PCM['Te'].values[roi_begin_idx:roi_end_idx])

## Compare Results

### 4 Order

In [ ]:
plt.figure()
plt.plot(ts_dpsim_4VBR['Te'].time, ts_dpsim_4VBR['Te'].values, label='VBR')
plt.plot(ts_dpsim_4PCM['Te'].time, ts_dpsim_4PCM['Te'].values, linestyle='--', label='4 Order PCM - MaxIter 10')
plt.legend()
plt.show()

In [ ]:
rmse = ts_dpsim_4VBR_roi['Te'].rmse(ts_dpsim_4PCM_roi['Te'], ts_dpsim_4VBR_roi['Te'])/te_ref*100
print('RMSE of PCM 4 Order = {}%'.format(rmse))
assert(rmse<0.449)

### 6 Order

In [ ]:
plt.figure()
plt.plot(ts_dpsim_6VBR['Te'].time, ts_dpsim_6VBR['Te'].values, label='6 Order VBR')
plt.plot(ts_dpsim_6PCM['Te'].time, ts_dpsim_6PCM['Te'].values, linestyle='--', label='6 Order PCM - MaxIter 10')
plt.legend()
plt.show()

In [ ]:
rmse = ts_dpsim_6VBR_roi['Te'].rmse(ts_dpsim_6PCM_roi['Te'], ts_dpsim_6VBR_roi['Te'])/te_ref*100
print('RMSE of PCM 6 Order = {}%'.format(rmse))
assert(rmse<0.381)